In [11]:
import pandas as pd
import math

# importing files
courier_invoice = pd.read_excel("https://github.com/DhivvyanThiyagu/Cointab-DataAnalyst-Assignment/blob/main/Courier%20Company%20-%20Invoice.xlsx?raw=true")
x_order_rpt = pd.read_excel("https://github.com/DhivvyanThiyagu/Cointab-DataAnalyst-Assignment/blob/main/Company%20X%20-%20Order%20Report.xlsx?raw=true")
x_pin_zone = pd.read_excel("https://github.com/DhivvyanThiyagu/Cointab-DataAnalyst-Assignment/blob/main/Company%20X%20-%20Pincode%20Zones.xlsx?raw=true")
x_sku = pd.read_excel("https://github.com/DhivvyanThiyagu/Cointab-DataAnalyst-Assignment/blob/main/Company%20X%20-%20SKU%20Master.xlsx?raw=true")
courier_charges = pd.read_excel("https://github.com/DhivvyanThiyagu/Cointab-DataAnalyst-Assignment/blob/main/Courier%20Company%20-%20Rates.xlsx?raw=true")


a = pd.merge(x_order_rpt,x_sku,on ="SKU")
a["Total Weight as per X"] = ((a["Order Qty"]*a["Weight (g)"])/1000)

b = pd.pivot_table(a,index = "ExternOrderNo",aggfunc = "sum")
b= b.reset_index()
b.drop(["Weight (g)","Order Qty"],axis =1, inplace = True)
b.columns = ["Order ID","Total Weight as per X (KG)"]

courier_invoice = pd.merge(courier_invoice,b, on="Order ID" )


def calculate_weight_slab(weight):
    return ((math.ceil(weight/0.5))/2)

courier_invoice["Weight_X"] = courier_invoice["Total Weight as per X (KG)"].apply(calculate_weight_slab)

courier_invoice["Weight slab charged by Courier Company (KG)"] = courier_invoice["Charged Weight"].apply(calculate_weight_slab)


courier_invoice.rename(columns = {'Zone':'Delivery Zone charged by Courier Company'}, inplace = True)
courier_invoice = pd.merge(courier_invoice,x_pin_zone,on ="Customer Pincode"	)


def charges(Zone, Type_of_Shipment, Weight_X):

  if Type_of_Shipment == "Forward charges":
    if Zone  == "a":
        if Weight_X / 0.5 == 0.5:
          return 29.5
        else: 
          a = Weight_X /0.5   
          return (29.5 + (23.6*(a-1)))

    elif Zone  == "b":
        if Weight_X / 0.5 == 0.5:
          return 33.0
        else: 
          a = Weight_X /0.5   
          return (33+ (28.3*(a-1)))

    elif Zone  == "c":
        if Weight_X / 0.5 == 0.5:
          return 40.1
        else: 
          a = Weight_X /0.5   
          return (40.1+ (38.9*(a-1)))

    elif Zone  == "d":
        if Weight_X / 0.5 == 0.5:
          return 45.4
        else: 
          a = Weight_X /0.5   
          return (45.4+ (44.8*(a-1)))

    elif Zone  == "e":
        if Weight_X / 0.5 == 0.5:
          return 56.6
        else: 
          a = Weight_X /0.5   
          return (56.6+ (55.5*(a-1)))

  elif Type_of_Shipment == "Forward and RTO charges":
    if Zone  == "a":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 13.6)
        else: 
          a = Weight_X /0.5   
          return (29.5 + 13.6 + ((23.6*(a-1)*2)))

    elif Zone  == "b":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 20.5)
        else: 
          a = Weight_X /0.5   
          return (29.5 + 20.5 + ((23.6*(a-1)*2)))

    if Zone  == "c":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 31.9)
        else: 
          a = Weight_X /0.5   
          return (29.5 + 31.9 + ((23.6*(a-1)*2)))

    if Zone  == "d":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 41.3)
        else: 
          a = Weight_X /0.5   
          return (29.5 + 41.3 + ((23.6*(a-1)*2)))

    if Zone  == "e":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 55.5)
        else: 
          a = Weight_X /0.5   
          return (29.5 + 55.5 + ((23.6*(a-1)*2)))



courier_invoice['Expected Charge as per X (Rs)'] = courier_invoice.apply(lambda row: charges(row['Zone'], row['Type of Shipment'], row['Weight_X']), axis=1)

courier_invoice.drop_duplicates()

courier_invoice["Difference Between Expected Charges and Billed Charges (Rs.)"]=courier_invoice["Expected Charge as per X (Rs)"]-courier_invoice["Billing Amount (Rs.)"]

courier_invoice.rename(columns={"AWB Code":"AWB Number","Charged Weight":"Total weight as per Courier Company (KG)","Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.) " ,"Zone":"Delivery Zone as per X","Weight_X":"Weight slab as per X (KG)"},inplace=True)

courier_invoice.drop(columns=["Warehouse Pincode_y","Type of Shipment","Customer Pincode","Warehouse Pincode_x"],axis=1,inplace=True)

courier_invoice.reindex(columns=["Order ID","AWB Number","Total weight as per X (KG)","Weight slab as per X (KG)","Total weight as per Courier Company (KG)","Weight slab charged by Courier Company (KG)","Delivery Zone as per X","Delivery Zone charged by Courier Company","Expected Charge as per X (Rs.)","Charges Billed by Courier Company (Rs.)"])



courier_invoice.to_csv("Result.csv", index=False)

courier_invoice




,AWB Number,Order ID,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Total Weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs),Difference Between Expected Charges and Billed Charges (Rs.)
0,1091117222124,2001806232,1.30,d,135.0,1.302,1.5,1.5,d,135.0,0.000000e+00
1,1091117222194,2001806273,1.00,d,90.2,0.615,1.0,1.0,d,90.2,-1.421085e-14
2,1091117222931,2001806408,2.50,d,224.6,2.265,2.5,2.5,d,224.6,0.000000e+00
3,1091117223244,2001806458,1.00,b,61.3,0.700,1.0,1.0,b,61.3,0.000000e+00
4,1091117229345,2001807012,0.15,d,45.4,0.240,0.5,0.5,d,45.4,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
169,1091121846136,2001811305,0.50,d,45.4,0.750,1.0,0.5,b,61.3,1.590000e+01
170,1091121846136,2001811305,0.50,d,45.4,0.750,1.0,0.5,b,61.3,1.590000e+01
171,1091118551656,2001812941,0.73,d,90.2,0.500,0.5,1.0,b,33.0,-5.720000e+01
172,1091117614452,2001809383,0.50,d,86.7,0.607,1.0,0.5,b,97.2,1.050000e+01
